In [2]:
import sys
import os
import pandas as pd
import plotly.express as px

sys.path.append(os.path.abspath(".."))
from db import run_query

## 1. Знакомство с данными

В проекте используются следующие таблицы:
- `characters` — список персонажей (140 строк)
- `spells` — список заклинаний (301 строка)
- `potions` — список зелий (72 строки)
- `quotes` — реплики из трёх фильмов (около 5000 строк)

Мы будем анализировать реплики и поведение персонажей, их связь с магией, а также частотность заклинаний.

In [3]:
run_query("SELECT * FROM characters LIMIT 5")

,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,None
1,2,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,None
2,3,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",None
3,4,Albus Percival Wulfric Brian Dumbledore,Male,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,Silver| formerly auburn,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,Late August 1881,"30 June, 1997"
4,5,Rubeus Hagrid,Male,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",None,Half-Human/Half-Giant,Part-Human (Half-giant),Black,Black,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,6 December 1928,None


In [4]:
tables = ["characters", "spells", "potions", "quotes"]

for table in tables:
    query = f"SELECT COUNT(*) AS rows FROM {table}"
    df = run_query(query)
    print(f"{table}: {df.rows[0]} строк")

    df_cols = run_query(f"PRAGMA table_info('{table}')")
    print(f"{table}: {len(df_cols)} столбцов\n")

characters: 140 строк
characters: 15 столбцов

spells: 301 строк
spells: 5 столбцов

potions: 72 строк
potions: 5 столбцов

quotes: 4925 строк
quotes: 4 столбцов



In [5]:
for table in tables:
    print(f"Таблица: {table}")
    display(run_query(f"SELECT * FROM {table} LIMIT 5"))

Таблица: characters


,Id,Name,Gender,Job,House,Wand,Patronus,Species,Blood status,Hair colour,Eye colour,Loyalty,Skills,Birth,Death
0,1,Harry James Potter,Male,Student,Gryffindor,"11"" Holly phoenix feather",Stag,Human,Half-blood,Black,Bright green,Albus Dumbledore | Dumbledore's Army | Order o...,Parseltongue| Defence Against the Dark Arts | ...,31 July 1980,None
1,2,Ronald Bilius Weasley,Male,Student,Gryffindor,"12"" Ash unicorn tail hair",Jack Russell terrier,Human,Pure-blood,Red,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Wizard chess | Quidditch goalkeeping,1 March 1980,None
2,3,Hermione Jean Granger,Female,Student,Gryffindor,"10¾"" vine wood dragon heartstring",Otter,Human,Muggle-born,Brown,Brown,Dumbledore's Army | Order of the Phoenix | Hog...,Almost everything,"19 September, 1979",None
3,4,Albus Percival Wulfric Brian Dumbledore,Male,Headmaster,Gryffindor,"15"" Elder Thestral tail hair core",Phoenix,Human,Half-blood,Silver| formerly auburn,Blue,Dumbledore's Army | Order of the Phoenix | Hog...,Considered by many to be one of the most power...,Late August 1881,"30 June, 1997"
4,5,Rubeus Hagrid,Male,Keeper of Keys and Grounds | Professor of Care...,Gryffindor,"16"" Oak unknown core",None,Half-Human/Half-Giant,Part-Human (Half-giant),Black,Black,Albus Dumbledore | Order of the Phoenix | Hogw...,Resistant to stunning spells| above average st...,6 December 1928,None


Таблица: spells


,Name,Incantation,Type,Effect,Light
0,Summoning Charm,Accio,Charm,Summons an object,None
1,Age Line,Unknown,Charm,Prevents people above or below a certain age f...,Blue
2,Water-Making Spell,Aguamenti,"Charm, Conjuration",Conjures water,Icy blue
3,Launch an object up into the air,Alarte Ascendare,Charm,Rockets target upward,Red
4,Albus Dumbledore's Forceful Spell,Unknown,Spell,Great Force,None


Таблица: potions


,Name,Known ingredients,Effect,Characteristics,Difficulty level
0,Ageing Potion,"Newt spleens , Bananas",Ages drinker temporarily,Green,Advanced
1,Amortentia,None,Love Potion that causes a powerful infatuation...,"Mother-of-pearl sheen, Spiralling steam, Scent...",Advanced
2,Antidote to Veritaserum,None,Counters the effect of Veritaserum,None,None
3,Babbling Beverage,"Valerian sprigs, Aconite, Dittany",Causes the drinker to speak nonsense,None,None
4,Baruffio's Brain Elixir,"Leaping Toadstools, Frog Brains, Runespoor egg...",Allegedly increases one's brain power,Green in colour,None


Таблица: quotes


,Character,Sentence,CHARACTER_1,SENTENCE_1
0,Dumbledore,"I should've known that you would be here, Prof...",None,None
1,McGonagall,"Good evening, Professor Dumbledore.",None,None
2,McGonagall,"Are the rumors true, Albus?",None,None
3,Dumbledore,"I'm afraid so, professor.",None,None
4,Dumbledore,The good and the bad.,None,None


In [6]:
for table in tables:
    print(f"Типы данных в таблице: {table}")
    df = run_query(f"PRAGMA table_info('{table}')")
    display(df[["name", "type"]])

Типы данных в таблице: characters


,name,type
0,Id,BIGINT
1,Name,VARCHAR
2,Gender,VARCHAR
3,Job,VARCHAR
4,House,VARCHAR
5,Wand,VARCHAR
6,Patronus,VARCHAR
7,Species,VARCHAR
8,Blood status,VARCHAR
9,Hair colour,VARCHAR


Типы данных в таблице: spells


,name,type
0,Name,VARCHAR
1,Incantation,VARCHAR
2,Type,VARCHAR
3,Effect,VARCHAR
4,Light,VARCHAR


Типы данных в таблице: potions


,name,type
0,Name,VARCHAR
1,Known ingredients,VARCHAR
2,Effect,VARCHAR
3,Characteristics,VARCHAR
4,Difficulty level,VARCHAR


Типы данных в таблице: quotes


,name,type
0,Character,VARCHAR
1,Sentence,VARCHAR
2,CHARACTER_1,VARCHAR
3,SENTENCE_1,VARCHAR


In [7]:
for table in tables:
    print(f"Пропущенные значения — {table}")
    df = run_query(f"SELECT * FROM {table}")
    nulls = df.isna().mean().sort_values(ascending=False) * 100
    display(nulls[nulls > 0])

Пропущенные значения — characters


Death           70.000000
Eye colour      38.571429
Loyalty         36.428571
House           27.857143
Skills          19.285714
Job             13.571429
Patronus        12.142857
Blood status    12.142857
Hair colour     12.142857
Birth            9.285714
Wand             5.714286
Gender           0.714286
dtype: float64

Пропущенные значения — spells


Light          51.495017
Incantation     2.325581
dtype: float64

Пропущенные значения — potions


Difficulty level     65.277778
Characteristics      43.055556
Known ingredients    41.666667
Effect                5.555556
dtype: float64

Пропущенные значения — quotes


SENTENCE_1     66.741117
CHARACTER_1    66.741117
Sentence       33.258883
Character      33.258883
dtype: float64

In [8]:
for table in tables:
    df = run_query(f"SELECT * FROM {table}")
    duplicates = df.duplicated().sum()
    print(f"Дубликатов в {table}: {duplicates}")

Дубликатов в characters: 0
Дубликатов в spells: 0
Дубликатов в potions: 0
Дубликатов в quotes: 91


In [9]:
import numpy as np

for table in tables:
    df = run_query(f"SELECT * FROM {table}")
    num_cols = df.select_dtypes(include=[np.number]).columns
    if len(num_cols) > 0:
        print(f"Выбросы — {table}")
        for col in num_cols:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            outliers = df[(df[col] < lower) | (df[col] > upper)]
            print(f"{col}: {len(outliers)} выбросов")

Выбросы — characters
Id: 0 выбросов


In [10]:
run_query("""
SELECT House, COUNT(*) AS num_characters
FROM characters
WHERE House IS NOT NULL AND House <> ''
GROUP BY House
ORDER BY num_characters DESC
""")


,House,num_characters
0,Gryffindor,38
1,Slytherin,28
2,Ravenclaw,18
3,Hufflepuff,13
4,Beauxbatons Academy of Magic,3
5,Durmstrang Institute,1


In [11]:
run_query("""
SELECT 
    Character,
    AVG(LENGTH(Sentence)) AS avg_length
FROM quotes
GROUP BY Character
HAVING COUNT(*) > 20
ORDER BY avg_length DESC
LIMIT 10
""")

,Character,avg_length
0,Snape,61.954545
1,Dumbledore,53.297872
2,TOM RIDDLE,48.428571
3,DUMBLEDORE,46.625000
4,MCGONAGALL,46.606061
5,MOANING MYRTLE,45.240000
6,SNAPE,45.148148
7,Madam Hooch,42.166667
8,McGonagall,38.521739
9,Hagrid,36.469799


In [12]:
run_query("""
SELECT 
    c.House,
    COUNT(*) AS num_quotes
FROM quotes q
JOIN characters c ON q.Character = c.Name
WHERE c.House IS NOT NULL AND c.House <> ''
GROUP BY c.House
ORDER BY num_quotes DESC
""")

,House,num_quotes


In [13]:
run_query("""
SELECT 
    s.Name AS spell_name,
    COUNT(*) AS mentions
FROM quotes q
JOIN spells s ON LOWER(q.Sentence) LIKE '%' || LOWER(s.Name) || '%'
GROUP BY s.Name
ORDER BY mentions DESC
LIMIT 10
""")


,spell_name,mentions
0,Pack,3
1,Vera Verto,3
2,Memory Charm,2
3,Oculus Reparo,2
4,Cistem Aperio,1
5,Peskipiksi Pesternomi,1
6,Everte Statum,1
7,Leg-Locker Curse,1
8,Brackium Emendo,1


In [14]:
run_query("""
SELECT COUNT(DISTINCT Job) AS unique_jobs
FROM characters
WHERE Job IS NOT NULL AND Job <> ''
""")

,unique_jobs
0,65


In [15]:
run_query("""
SELECT 
    CASE 
        WHEN c.Patronus IS NULL OR c.Patronus = '' THEN 'Нет'
        ELSE 'Есть'
    END AS has_patronus,
    COUNT(*) AS total_quotes
FROM quotes q
JOIN characters c ON q.Character = c.Name
GROUP BY has_patronus
""")

,has_patronus,total_quotes


In [16]:
run_query("""
SELECT Character, COUNT(*) AS num_quotes
FROM quotes
GROUP BY Character
ORDER BY num_quotes DESC
LIMIT 10
""")

,Character,num_quotes
0,None,1638
1,HARRY,368
2,RON,165
3,Harry,155
4,Hagrid,149
5,Harry,136
6,HERMIONE,122
7,GILDEROY LOCKHART,113
8,Ron,104
9,DUMBLEDORE,88


In [17]:
df = run_query("""
SELECT House, COUNT(*) AS num_characters
FROM characters
WHERE House IS NOT NULL AND House <> ''
GROUP BY House
ORDER BY num_characters DESC
""")

import plotly.express as px
px.bar(df, x="House", y="num_characters", title="Количество персонажей по факультетам")

In [18]:
df = run_query("""
SELECT LENGTH(Sentence) AS sentence_length
FROM quotes
WHERE LENGTH(Sentence) < 300
""")

px.histogram(df, x="sentence_length", nbins=30, title="Распределение длины реплик (до 300 символов)")

In [20]:
df = run_query("""
SELECT 
    Character,
    COUNT(*) AS num_quotes,
    AVG(LENGTH(Sentence)) AS avg_length
FROM quotes
GROUP BY Character
HAVING num_quotes > 15
""")

px.scatter(df, x="num_quotes", y="avg_length", hover_name="Character", size="num_quotes",
           title="Средняя длина vs Количество реплик (персонажи с >15 фраз)")

In [ ]:
df = run_query("""
SELECT 
    s.Name AS spell_name,
    COUNT(*) AS mentions
FROM quotes q
JOIN spells s ON LOWER(q.Sentence) LIKE '%' || LOWER(s.Name) || '%'
GROUP BY s.Name
ORDER BY mentions DESC
LIMIT 10
""")

import plotly.express as px
px.bar(df, x="spell_name", y="mentions", title="Топ-10 заклинаний по упоминаниям в репликах")

In [ ]:
## Гипотеза 1: Гермиона говорит больше, чем Рон

df = run_query("""
SELECT Character, COUNT(*) AS num_quotes
FROM quotes
WHERE Character IN ('Hermione', 'Ron')
GROUP BY Character
""")

import plotly.express as px
px.bar(df, x="Character", y="num_quotes", title="Сравнение количества реплик Гермионы и Рона")

In [ ]:
## Гипотеза 2: Дамблдор говорит длиннее всех из главных персонажей

df = run_query("""
SELECT 
    Character,
    AVG(LENGTH(Sentence)) AS avg_length
FROM quotes
WHERE Character IN ('Harry', 'Hermione', 'Ron', 'Dumbledore', 'Voldemort')
GROUP BY Character
""")

import plotly.express as px
px.bar(df, x="Character", y="avg_length", title="Средняя длина реплик у главных персонажей")

In [27]:
## Гипотеза 3: Самые частые заклинания относятся к атакующему типу

df = run_query("""
SELECT Type, COUNT(*) AS num_spells
FROM spells
WHERE Type IS NOT NULL AND Type <> ''
GROUP BY Type
ORDER BY num_spells DESC
""")

import plotly.express as px
px.bar(df, x="Type", y="num_spells", title="Частота заклинаний по типу")